### Calulcation of Atmospheric dispersion is integrated with different cases

In [1]:
import heeps
from heeps.util.coord import disk_coord, mas2rms
import numpy as np
from astropy.io import fits 
from heeps.util.save2fits import save2fits
from heeps.util.notify import notify
from heeps.util.coord import mas2rms
# Gets the dispersion values for HCI L-long band and transit parameters for Alpha-cen case
from heeps.util.get_atm_disp_vals import get_disp_vals, get_transit_para_alpha_cen

In [2]:
cases = [#'ideal']
         #'scao_only']
         #'apo_drift',
         #'talbot/qsta',
         #'talbot/static']
         #'cbw_ncpa',
         #'wv_ncpa',
         #'all_ncpa',
         'all_effects']
         #'all_effects_misseg']
         #'water_vapor/scao_only/yes',
         #'water_vapor/scao_only/WVnoTT']
         #'water_vapor_10min/scao_only']
         #'water_vapor_10min/custom',
         #'water_vapor_10min/kolmog']

In [3]:
#lag=0
#for ctrl in ['G_0.4_nLF_10_lag_%s'%lag,
#             'G_0.4_nLF_10_nHF_10_lag_%s'%lag,
#             'G_0.4_nLF_10_nHF_3_lag_%s'%lag,
#             'G_0.4_nLF_10_nHF_1_lag_%s'%lag]:
# cases = ['water_vapor/scao_only/%s'%ctrl]
#for rms in [10,20,50,100]:
# cases = ['noTTnoPP/petal_piston_LTF%s'%rms,
#          'noTTnoPP/petal_piston_HTF%s'%rms]
#for rms in [0.01,0.02,0.03]:
# cases = ['scao_only/seg_refl%s'%rms]
#for apod in [1, 2, 3, 5]:
# cases = ['scao_only/apo_drift%s'%apod]
#for rep in ['rep_1_-0h15','rep_1_-1h00','rep_1_-2h00']:
# cases = ['talbot/%s'%rep]
#for leak in [1e-3, 2e-3, 5e-3, 1e-2, 2e-2]:
#for leak in [5e-3, 1e-2, 2e-2, 5e-2, 1e-1]:
#for leak in [5e-2]:
# cases = ['chrom_leak/%.0e'%leak]

In [4]:
for case in cases:
    print('\nCase = %s'%case)
    scao_case = 'all_ncpa' if 'all_ncpa' in case or 'all_effects' in case \
           else 'noTTnoPP' if 'noTTnoPP' in case else 'scao_only'

    # 1. Create a config dictionary with your simulation parameters.
    #    Undefined parameters get default values from calling read_config
    conf = dict(
        bands = ['L'],#,'N2'],#'M','N2','N2a'],
        modes = ['RAVC'],#'CLC', 'RAVC'],
        
        nframes = 12,
        #nstep = 3,
        cpu_count = 28,
        dir_output = '/mnt/disk12tb/Users/ppathak/final_atmospheric_disp_calculation/%s'%case,
        send_to = 'ppathak@uliege.be',

        f_pupil = 'pupil/ELT_fullM1_refl_0.02rms_misseg_1x7.fits' if 'misseg' in case else
                  'pupil/ELT_fullM1_refl_0.02rms.fits' if 'all_effects' in case or 'seg_refl' in case else
                  'pupil/ELT_fullM1.fits',
 #       f_pupil = 'pupil/ELT_fullM1_refl_%srms.fits'%rms,
 #       f_pupil = 'pupil/ELT_fullM1_misseg_%sx7.fits'%nmis,
        
        add_phase = False if 'ideal' in case else True,
        f_phase = 'wavefront/cfull/wv/cube_Cfull_20220512_3600s_300ms_0piston_meters_%s'%scao_case + '_%s_%s_WVnoTT.fits' if 'WVnoTT' in case
             else 'wavefront/cfull/wv/cube_Cfull_20220512_3600s_300ms_0piston_meters_%s'%scao_case + '_%s_%s_WV.fits' if 'yes' in case
             else 'wavefront/cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_cbw_ncpa_%s_%s.fits' if 'cbw' in case
             else 'wavefront/cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_wv_ncpa_%s_%s.fits' if 'wv' in case
             # else 'wavefront/cfull/old_cbw_ncpa/cube_Cfull_20220512_3600s_300ms_0piston_meters_%s'%scao_case + '_%s_%s.fits',
             else 'wavefront/cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_%s'%scao_case + '_%s_%s.fits',
        #f_phase = 'wavefront/cfull/cube_Cfull_20211004_600s_100ms_0piston_meters_scao_only_%s_%s.fits',
        #f_phase = 'wavefront/cfull/wv/cube_Cfull_20220512_3600s_300ms_0piston_meters_%s'%scao_case + '_%s_%s' + '_WV_%s.fits'%ctrl,
        #f_phase = 'wavefront/petal/cube_Cfull_20220512_3600s_300ms_0piston_meters_%s'%scao_case + '_%s_%s' + '_ppLTF_rms_%s.fits'%rms if 'LTF' in case
        #     else 'wavefront/petal/cube_Cfull_20220512_3600s_300ms_0piston_meters_%s'%scao_case + '_%s_%s' + '_ppHTF_rms_%s.fits'%rms,
        add_point_err = 'point' in case,# or 'all_effects' in case,
        f_point_err = 'wavefront/point/point_drift_3600s_300ms_%s.fits' if 'point_drift' in case else
                      'wavefront/point/point_jitter_3600s_300ms_%s.fits' if 'point_jitter' in case else
                      'wavefront/point/point_planet_3600s_300ms_%s.fits' if 'point_planet' in case else
                      'wavefront/point/point_all_3600s_300ms_%s.fits',
        #f_point_err = 'wavefront/point/point_err_0.01-0.1Hz_3600s_300ms_%s' + '_rms_%s.fits'%rms,
        #f_point_err = 'wavefront/point/point_err_0.1-1Hz_3600s_300ms_%s' + '_rms_%s.fits'%rms,
        add_apo_drift = 'apo_drift' in case or 'all_effects' in case,
        apo_drift = 0.02, # (% ptv)
        #apo_drift = apod/100,
        add_amp = 'talbot' in case or 'all_effects' in case,
        f_amp = 'wavefront/cbw/20220223/talbot/%s' + '_%s.fits'
            %('_rep_1_field_50' if 'static' in case else 'rep_1_-0h30'),# else rep),
        add_cl_vort = 'chrom' in case or 'all_effects' in case,
        # alpha Cen = 4.26 mas ; pi1 Gru = 9.18 mas
        fp_offsets = mas2rms(disk_coord(4.26, nr=4), 36.905) if 'alphaCen' in case else
                     mas2rms(disk_coord(9.18, nr=4), 36.905) if 'pi1Gru' in case else
                     None,
        f_app_amp = 'optics/app/METIS_%s_%s_amp_ROF.fits',
        f_app_phase = 'optics/app/METIS_%s_%s_phase_ROF.fits',
        )
    
    # read config
    conf = heeps.config.read_config(verbose=False, **conf)
    f_phase = conf['f_phase']
    f_amp = conf['f_amp']
    f_point_err = conf['f_point_err']
    f_app_amp = conf['f_app_amp']
    f_app_phase = conf['f_app_phase']

    
    # Gets Alpha-cen 6hrs trasit zenith angles, used for dispersion calc
    star_zenith = get_transit_para_alpha_cen(**conf)
    adc = 40 # ADC optimized for a zenith angle [Integer]  
    steps = 5 # sampling of HCI L-long band
    
    # HCI L-long wavelengths for the given sampling, residual disp fit in x & y axis, 
    # and atmospheric disp in y-axis
    sel_L_long, fit_res_x, fit_res_y, fit_disp_y = get_disp_vals(adc, steps, **conf)
    
    # from 6hrs transit to 4hrs, the 6hrs trasnit is sampled into 600 steps
    cut_zen = star_zenith[100:500]
    
    # using the above transit and fitting it, so that zenith angle is calculated for given number of frames
    x = np.linspace(0, 4, cut_zen.shape[0])
    fit = np.polyfit(x, cut_zen, 5)
    x1 = np.linspace(0, 4, conf['nframes'])
    new_zen = np.polyval(fit, x1)

    for conf['band'] in conf['bands']:
        nband = heeps.config.update_config(**dict(heeps.config.read_config(), band=conf['band']))['npupil']

        conf['f_phase'] = f_phase%(conf['band'], nband)
        conf['f_amp'] = f_amp%('N' if 'N' in conf['band'] else 'LM')
        conf['f_point_err'] = f_point_err%conf['band']
        
        #conf['hfov'] = 1.1 if conf['band'] in ['L' , 'M'] else 1.3
        conf['hfov'] = 0.8 if conf['band'] in ['L' , 'M'] else 1.3
        #conf['hfov'] = 1.23

        for conf['mode'] in conf['modes']:
            
            if 'APP' in conf['mode']:
                APP_case = 'LMS' if 'LMS' in conf['mode'] else 'IMG'
                conf['f_app_amp'] = f_app_amp%(APP_case, conf['band'])
                conf['f_app_phase'] = f_app_phase%(APP_case, conf['band'])

            if 'VC' in conf['mode']:
                conf['f_oat'] = conf['dir_input'] + 'optics/oat_%s_%s.fits'%(conf['band'], conf['mode'])

            if 'L' in conf['band'] or 'M' in conf['band']:
                leak = 2e-3
                conf['vc_chrom_leak'] = leak
                if 'RAVC' in conf['mode']:
                    conf.update(ls_dRext=0.0477, ls_dRint=0.04, ls_dRspi=0.0249)
                elif 'CVC' in conf['mode']:
                    conf.update(ls_dRext=0.0209, ls_dRint=0.09, ls_dRspi=0.0245)
                elif 'CLC' in conf['mode']:
                    conf.update(ls_dRext=0.1095, ls_dRint=0.0491, ls_dRspi=0.0317)

            elif 'N' in conf['band']:
                leak = 1e-2
                conf['vc_chrom_leak'] = leak
                if 'CVC' in conf['mode']:
                    conf.update(ls_dRext=0.0268, ls_dRint=0.09, ls_dRspi=0.0357)
                elif 'CLC' in conf['mode']:
                    conf.update(ls_dRext=0.1045, ls_dRint=0.0511, ls_dRspi=0.0383)

            # update conf without verbose, for ADI only
            conf = heeps.config.update_config(saveconf=False, verbose=False, **conf) 

            if True:
                # Stores the polychromatic PSFs for a given nu of frames
                psfs = np.zeros((steps, conf['nframes'], conf['ndet'], conf['ndet']), dtype=np.float32)
                # Polychromatic off-axis PSF
                psfs1 = np.zeros((steps, conf['ndet'], conf['ndet']), dtype=np.float32)
                for i, lam in enumerate(sel_L_long*1e-6):
                    conf['lam'] = lam
                    # Atmsopheric dispersion
                    tipx = np.polyval(fit_disp_y[i], new_zen)*0
                    tipy = np.polyval(fit_disp_y[i], new_zen)
                    # Residual dispersion (atmospheric+ADC)
                    # tipx = np.polyval(fit_res_x[i], new_zen)
                    # tipy = np.polyval(fit_res_y[i], new_zen)
                    conf['disp'] = True
                    print('DISP VALS', tipx, tipy)
                    
                    # converts mas values to RMS
                    tiptilts = np.array([mas2rms(tipx, conf['diam_ext']), mas2rms(tipy, conf['diam_ext'])]).T
                    # Saves the tip-tilt values for all the zenith values/nframes to be used during multiprocessing
                    fits.writeto(conf['dir_temp']+'dispersion_vals.fits', tiptilts , overwrite=True)
                    
                   # 2. Update config parameters. The following parameters
                    # will be updated to match the selected spectral band and HCI mode:
                    #   lam, pscale, flux_star, flux_bckg, ls_dRspi, ls_dRint, npupil, ndet,
                    #   ravc_t, ravc_r
                    conf = heeps.config.update_config(saveconf=True, verbose=True, **conf)                
                    print(i,conf['lam'], conf['npupil'])
                    # 3. Load entrance pupil, and create 'wavefront' object
                    wf = heeps.pupil.pupil(savefits=False, verbose=True, **conf)

                    # 4. Create an off-axis PSF template for injecting fake exoplanets
                    conf['nframes_avg'] = min(100, conf['nframes'])
                    psfs1[i] = heeps.wavefront.propagate(wf, onaxis=False, avg=True, savefits=False, verbose=False, **conf);

                    # 5. Create a cube of on-axis PSFs (=star)
                    psfs[i] = heeps.wavefront.propagate(wf, onaxis=True, savefits=False, verbose=False, **conf);
                    print('FINAL PSF SHAPE',psfs.shape)   

                polypsf = np.mean(psfs, axis = 0) # wavelength axis gets averaged
                polypsf1 = np.mean(psfs1, axis = 0)
                
                tag = None
                tag = '' if tag is None else '%s_'%tag
                onaxis = False
                name = '%s%s_PSF'%(tag, {True: 'onaxis', False: 'offaxis'}[onaxis])
                filename = save2fits(polypsf1, name, **conf)

                onaxis = True
                name = '%s%s_PSF'%(tag, {True: 'onaxis', False: 'offaxis'}[onaxis])
                filename = save2fits(polypsf, name, **conf)
                notify('saved to %s'%filename, conf['send_to'])

            # 7. Produce 5-sigma sensitivity (contrast) curves for each set of PSFs (modes, bands)
            if True and not ('neg' in conf['mode'] or 'pos' in conf['mode'] or 'ideal' in case):
                conf['add_bckg'] = False
                if True:
                    conf['mag'] = -1.6 if 'N' in conf['band'] else 6
                    sep, adi = heeps.contrast.cc_adi(tag=case, savepsf=False,
                        savefits=True, verbose=True, nscreens=None, **conf)
                else:
                    ndets = {'L':329, 'M':329, 'N2':383, 'N2a':241} # LM 0.9", N2 1.3"
                    conf['ndet'] = ndets[conf['band']]
                    mags = {
                        'L':  [-1.518, -1.38, -0.678, -0.63, 0.211, 1.652],
                        'M':  [-1.468, -1.344, -0.648, -0.566, 0.241, 1.709],
                        'N2': [-1.478],#, -1.312, -0.633, -0.592, 0.267, 1.691],
                        'N2a':[-1.478],#, -1.312, -0.633, -0.592, 0.267, 1.691],
                    }
                    for conf['mag'] in mags[conf['band']]:
                        sep, adi = heeps.contrast.cc_adi(tag=case, savepsf=False,
                            savefits=True, verbose=True, **conf)




Case = all_effects
fplane_img 403
fplane_img 293
DISP VALS [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [2.52119411 2.3763026  2.25980069 2.17217798 2.1136711  2.0843941
 2.0843941  2.1136711  2.17217798 2.25980069 2.3763026  2.52119411]
Simulation config: band=L, mode=RAVC
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾
fplane_img 293
   loading vortex back-propagation params
   vc_charge=2, ngrid=1024, diam_norm=1.0838, beam_ratio=0.2640
   ravc_calc=False, ravc_t=0.7909, ravc_r=0.5190
   npupil=293, pscale=5.4700 mas, lam=3.7072E-06 m
   hfov=0.8 arcsec (-> ndet=293, 38.68 lam/D)

0 3.707161376668874e-06 293
Entrance pupil: spi_width=0.54 m, seg_width=1.45 m, seg_gap=0.004 m, seg_rms=0
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾
   2023-01-17 15:30:37, e2e simulation using 28 cores
   2023-01-17 15:30:41, completed in 4.84 seconds
   2023-01-17 15:30:51, e2e simulation using 28 cores
   2023-01-17 15:30:59, completed in 8.03 seconds
FINAL PSF SHAPE (5, 12, 293, 293)
DISP VALS [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] [1.32043645 1.24455176 1.183535

AttributeError: module 'vip_hci' has no attribute 'medsub'